In [1]:
import sqlite3
import pandas as pd

In [2]:
survey = pd.read_csv('survey.csv')
quiz = pd.read_csv('quiz.csv')
home_try_on = pd.read_csv('home_try_on.csv')
purchase = pd.read_csv('purchase.csv')

In [3]:
survey.head(1)

,question,user_id,response
0,1. What are you looking for?,005e7f99-d48c-4fce-b605-10506c85aaf7,Women's Styles


In [4]:
quiz.head(1)

,user_id,style,fit,shape,color
0,4e8118dc-bb3d-49bf-85fc-cca8d83232ac,Women's Styles,Medium,Rectangular,Tortoise


In [5]:
home_try_on.head(1)

,user_id,number_of_pairs,address
0,d8addd87-3217-4429-9a01-d56d68111da7,5 pairs,145 New York 9a


In [6]:
purchase.head(1)

,user_id,product_id,style,model_name,color,price
0,00a9dd17-36c8-430c-9d76-df49d4197dcf,8,Women's Styles,Lucy,Jet Black,150


In [7]:
conn = sqlite3.connect('funnels.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

# Create table - survey
c.execute('''
            CREATE TABLE survey
            ([question] text, [user_id] text, [response] text)
''')

c.execute('''
            CREATE TABLE quiz
            ([user_id] text, [style] text, [fit] text, [shape] text, [color] text)
''')

c.execute('''
            CREATE TABLE home_try_on
            ([user_id] text, [number_of_pairs] text, [address] text)
''')

c.execute('''
            CREATE TABLE purchase
            ([user_id] text, [product_id] interger, [style] text, [model_name] text, [color] text, [price] interger)
''')

conn.commit()

In [8]:
for i in ['survey', 'quiz', 'home_try_on', 'purchase']:
    read = pd.read_csv(i + '.csv')
    read.to_sql(i, conn, if_exists='append', index = False)

In [11]:
pd.read_sql('''
            SELECT question,
              COUNT(DISTINCT user_id) 
            FROM survey
            GROUP BY question;
''', con=conn)

,question,COUNT(DISTINCT user_id)
0,1. What are you looking for?,500
1,2. What's your fit?,475
2,3. Which shapes do you like?,380
3,4. Which colors do you like?,361
4,5. When was your last eye exam?,270


In [13]:
pd.read_sql('''
            WITH q AS(
                SELECT '1-quiz' AS stage, COUNT(DISTINCT user_id)
                FROM quiz),
            h AS(
                SELECT '2-home-try-on' AS stage, COUNT(DISTINCT user_id)
                FROM home_try_on),
            p AS(
                SELECT '3-purchase' AS stage, COUNT(DISTINCT user_id)
                FROM purchase)

            SELECT * FROM q
            UNION ALL
            SELECT * FROM h
            UNION ALL
            SELECT * FROM p;
''', con=conn)

,stage,COUNT(DISTINCT user_id)
0,1-quiz,1000
1,2-home-try-on,750
2,3-purchase,495


In [14]:
pd.read_sql('''
            WITH funnel AS(
              SELECT DISTINCT q.user_id,
                h.user_id IS NOT NULL AS 'is_home_try_on',
                h.number_of_pairs,
                p.user_id IS NOT NULL AS 'is_purchase'
              FROM quiz AS 'q'
              LEFT JOIN home_try_on AS 'h'
              ON q.user_id = h.user_id
              LEFT JOIN purchase AS 'p'
              ON q.user_id = p.user_id)

            SELECT number_of_pairs,
                SUM(is_home_try_on) AS 'num_home_try_on',
                SUM(is_purchase) AS 'num_purchase',
                1.0 * SUM(is_purchase)/ COUNT(number_of_pairs) AS 'purchase_rate'
            FROM funnel
            GROUP BY 1
            HAVING num_home_try_on > 1;
''', con=conn)

,number_of_pairs,num_home_try_on,num_purchase,purchase_rate
0,3 pairs,379,201,0.530343
1,5 pairs,371,294,0.792453
